# Welcome to This Jupyter example
You simply type stuff in the "In" boxes, and it the play button.  You can also it shift-enter to play the box.  The cell menu lets you click multiple boxes.   This sort of thing can be the future of our self-paced spark spark training.


In [4]:
val x = sc.cassandraTable("retail","products_by_id")

receipt_id bigint,
    scan_id timeuuid,
    credit_card_number bigint static,
    credit_card_type text static,
    product_id text,
    product_name text,
    quantity int,
    receipt_timestamp timestamp static,
    receipt_total decimal static,
    register_id int static,
    solr_query text,
    store_id int static,
    total decimal,
    unit_price decimal,


In [317]:
case class Receipts(
    receipt_id: String,
    scan_id: String,
    credit_card_number: Option[String],
    credit_card_type: Option[String],
    receipt_timestamp: Option[String],
    receipt_total: Option[String],
    store_id: Option[String]
    
)

 scan_id timeuuid,
    receipt_timestamp timestamp,
    receipt_total decimal,
    store_id int,


In [86]:
val receipts = sc.cassandraTable("retail","receipts").select("receipt_id","credit_card_number","scan_id","receipt_timestamp","receipt_total","store_id").as(Receipts)

In [87]:
receipts.count

Long = 351677

In [316]:
total_credit_card.take(20)

Array[(String, String, String, String, String, String)] = Array((1444803402917,4716605562641,23998a14-723b-11e5-b4bd-8b496c707234,2015-02-01 17:30:36+0000,2499.77,393), (1444802830615,373994855939263,ce93e753-7239-11e5-b4bd-8b496c707234,2015-02-01 19:30:29+0000,1329.8,227), (1444691779674,5310231762049760,3f4b7d45-7137-11e5-9cc1-8b496c707234,2015-02-02 01:23:49+0000,1989.72,154), (1444798582375,5579704434928179,ea4380f2-722f-11e5-b4bd-8b496c707234,2015-02-01 23:12:02+0000,1609.72,368), (1444802771407,4916549559527,ab5f9ae7-7239-11e5-b4bd-8b496c707234,2015-02-02 02:41:45+0000,619.83,123), (1444691684673,371964228082366,065ff4c3-7137-11e5-9cc1-8b496c707234,2015-02-01 16:58:09+0000,999.77,223), (1444691209220,4556768116938,eb367c12-7135-11e5-9cc1-8b496c707234,2015-02-01 23:57:31+00...

In [163]:
case class Customers(
    id: Option[String]
)

In [318]:
val customers = sc.cassandraTable("retail","customers").select("id").map(x => x.getString(0))

In [319]:
val customers_with_key = customers.zipWithIndex().map(_.swap)

In [320]:
customers_with_key.take(10).foreach(println)

(0,dbc3347c-b5e2-4dda-b13d-47c3dc627ad0)
(1,a8e06888-519f-4a74-8bad-06e0dfbc7183)
(2,8b427fb1-1649-4d68-91ac-5a487fac0f6f)
(3,d17b4671-5034-456b-9843-906e70a63dd6)
(4,a2032764-a809-4338-8c10-94e6afd89e20)
(5,e05ce9bf-aaa8-4f49-bc4f-6c9b8a688cc1)
(6,45140371-5ea7-4392-aa66-b8d69f001d84)
(7,f7596900-68a9-4452-a5a4-a2f811b3a755)
(8,4d743b08-9f06-4d29-a344-1955e4f11d8a)
(9,737a3465-8655-40c8-8df8-969e08f11f4f)


In [ ]:
x.take(2)


make a link between customer and receipt

In [321]:
val receipts_with_key = sc.cassandraTable("retail","receipts").select("receipt_id","scan_id","credit_card_number","credit_card_type","receipt_timestamp","receipt_total","store_id").as(Receipts).map(x => (((x.receipt_id + x.scan_id).hashCode() % 5000).abs.toLong, (x.receipt_id, x.scan_id, x.credit_card_number, x.credit_card_type, x.receipt_timestamp, x.receipt_total, x.store_id)))

In [322]:
receipts_with_key.take(10).foreach(println)
receipts_with_key.count()

(2397,(1444798652898,14506d47-7230-11e5-b4bd-8b496c707234,Some(372718046963499),Some(American Express),Some(2015-02-01 19:55:15+0000),Some(1249.8500000000001),Some(247)))
(4931,(1444798652898,145c5421-7230-11e5-b4bd-8b496c707234,Some(372718046963499),Some(American Express),Some(2015-02-01 19:55:15+0000),Some(1249.8500000000001),Some(247)))
(2906,(1444798652898,14697380-7230-11e5-b4bd-8b496c707234,Some(372718046963499),Some(American Express),Some(2015-02-01 19:55:15+0000),Some(1249.8500000000001),Some(247)))
(1923,(1444798652898,14881f1b-7230-11e5-b4bd-8b496c707234,Some(372718046963499),Some(American Express),Some(2015-02-01 19:55:15+0000),Some(1249.8500000000001),Some(247)))
(2519,(1444798638448,0bb44b71-7230-11e5-b4bd-8b496c707234,Some(5150065054555248),Some(MasterCard),Some(2015-02-01 22:03:46+0000),Some(919.9200000000001),Some(390)))
(3741,(1444798638448,0bc08078-7230-11e5-b4bd-8b496c707234,Some(5150065054555248),Some(MasterCard),Some(2015-02-01 22:03:46+0000),Some(919.9200000000001

Long = 351677

In [323]:
val receipts_by_customer = receipts_with_key.join(customers_with_key).map{ case (x,y) => y}.map(_.swap)

## Some Case Class Example with some markdown
Lets look at the stores table

In [264]:
receipts_by_customer.count()

Long = 351677

In [324]:
receipts_by_customer.map{case (k,(a,b,c,d,e,f,g)) => (k,a,b,c,d,e,f,g)}.saveToCassandra("retail","receipts_by_customer", SomeColumns("customer_id","receipt_id","scan_id","credit_card_number","credit_card_type","receipt_timestamp","receipt_total","store_id"))

Name: org.apache.spark.SparkException
Message: Job aborted due to stage failure: Task 1 in stage 263.0 failed 1 times, most recent failure: Lost task 1.0 in stage 263.0 (TID 309, localhost): java.io.IOException: Failed to write statements to retail.receipts_by_customer.
	at com.datastax.spark.connector.writer.TableWriter$$anonfun$write$1.apply(TableWriter.scala:164)
	at com.datastax.spark.connector.writer.TableWriter$$anonfun$write$1.apply(TableWriter.scala:139)
	at com.datastax.spark.connector.cql.CassandraConnector$$anonfun$withSessionDo$1.apply(CassandraConnector.scala:110)
	at com.datastax.spark.connector.cql.CassandraConnector$$anonfun$withSessionDo$1.apply(CassandraConnector.scala:109)
	at com.datastax.spark.connector.cql.CassandraConnector.closeResourceAfterUse(CassandraConnector.scala:139)
	at com.datastax.spark.connector.cql.CassandraConnector.withSessionDo(CassandraConnector.scala:109)
	at com.datastax.spark.connector.writer.TableWriter.write(TableWriter.scala:139)
	at com.da

In [424]:
case class R (
    customer_id: String,
    receipt_id: String,
    store_id: String,
    receipt_total: Option[Float]
)

In [425]:
val customer_receipt_store = sc.cassandraTable("retail","receipts_by_customer").select("customer_id","receipt_id","store_id","total").as(R)

val top_customers = customer_receipt_store.map(x => (x.customer_id, x.receipt_id, x.store_id)).map{case (a,b,c) => (a,(c,1))}.groupByKey()

In [426]:
val receipts_by_customers = sc.cassandraTable("retail","receipts_by_customer").select("customer_id","receipt_id","store_id","receipt_total").as(R)

val receipts_by_customers_c = receipts_by_customer.filter(x => x.receipt_total != null)

In [427]:
val total = receipts_by_customers.map( x => ((x.store_id, x.customer_id), x.receipt_total.getOrElse(0.0f))).reduceByKey(_+_)

In [430]:
val total2 = total.map{ case ((store_id, customer_id), total) => (store_id, (customer_id, total)) }

In [438]:
total2.groupByKey().take(4)(1)

(String, Iterable[(String, Float)]) = (273,CompactBuffer((eb6e2e45-088d-46e4-9fed-7428285acad8,939.87), (b34bbd01-4052-4bf7-9cfe-95f6571cb79b,909.85), (88fdc174-0fbc-4fd0-a37b-e5808fccdf26,1309.88), (da77aac3-42e1-4315-89da-63f9f21a0095,1589.86), (eacbd77e-3db4-4e7c-b20e-2fdceb83d541,2779.66), (940bc211-a8fe-4d82-add9-a8f55441a70a,1429.86), (014d8728-1465-4c38-abad-97e828b21d44,2779.66), (4728df9c-2ee6-4142-8133-23c8dc848e19,1259.77), (f3a78056-5b6b-4134-a7df-a95c87593e6a,209.93), (02180274-4a21-406b-96c0-ee75c1ba72cb,1189.9), (420f7355-b949-468d-8b0f-4692e34a7957,999.82), (1b681a02-ac4e-43ef-8476-fd6bcba469fb,919.88), (0f1a3f96-424d-45ce-8e0a-ceb4e68592a6,2639.67), (2eef0c99-c4ab-4709-ab9c-a154d292c1a0,1049.87), (2a622002-7122-4615-a8d7-6c6e00f24123,529.87), (787fad57-a49c-4d43...

In [359]:
import org.apache.spark.sql.SQLContext

val sqlContext = new SQLContext(sc)

val receipts_df = sqlContext.read.format("org.apache.spark.sql.cassandra").
    options(Map("keyspace"-> "retail", "table" -> "receipts")).
    load()
receipts_df.registerTempTable("receipts")

val receipts_by_customer = sqlContext.read.format("org.apache.spark.sql.cassandra").
    options(Map("keyspace"-> "retail", "table" -> "receipts_by_customer")).
    load()
    
    
receipts_by_customer.registerTempTable("receipts_by_customer")

val customers = sqlContext.read.format("org.apache.spark.sql.cassandra").
    options(Map("keyspace"-> "retail", "table" -> "customers")).
    load()
    
    
customers.registerTempTable("customers")

val x = sqlContext.sql("select c.firstname, c.lastname, r.store_id, sum(r.total) " +
    " from receipts r,receipts_by_customer rc, customers c "+
    " where r.receipt_id = rc.receipt_id and rc.customer_id = c.id group by c.firstname,c.lastname,r.store_id")
x.show



Name: org.apache.spark.SparkException
Message: Job aborted due to stage failure: Task 0 in stage 295.0 failed 1 times, most recent failure: Lost task 0.0 in stage 295.0 (TID 345, localhost): scala.MatchError: UUIDType (of class org.apache.spark.sql.cassandra.types.UUIDType$)
	at org.apache.spark.sql.execution.SparkSqlSerializer2$$anonfun$createSerializationFunction$1.apply(SparkSqlSerializer2.scala:232)
	at org.apache.spark.sql.execution.SparkSqlSerializer2$$anonfun$createSerializationFunction$1.apply(SparkSqlSerializer2.scala:227)
	at org.apache.spark.sql.execution.Serializer2SerializationStream.writeValue(SparkSqlSerializer2.scala:70)
	at org.apache.spark.storage.DiskBlockObjectWriter.write(BlockObjectWriter.scala:207)
	at org.apache.spark.util.collection.WritablePartitionedIterator$$anon$3.writeNext(WritablePartitionedPairCollection.scala:104)
	at org.apache.spark.util.collection.ExternalSorter.spillToPartitionFiles(ExternalSorter.scala:375)
	at org.apache.spark.util.collection.Exte

In [ ]:
case class Store(
                  store_id: Int,
                  address: String,
                  address_2: String,
                  address_3: String,
                  city: String,
                  state: String,
                  zip: Long,
                  size_in_sf: Int)

In [362]:
val stores_df = sqlContext.read.format("org.apache.spark.sql.cassandra").
        options(Map("keyspace"-> "retail", "table" -> "stores")).
        load().cache()
stores_df.registerTempTable("stores")

import org.apache.spark.sql.SaveMode

val init_potential_fraud_per_state_df = sqlContext.sql("select distinct s.state, 0 as number from  stores s "   )
init_potential_fraud_per_state_df.write.format("org.apache.spark.sql.cassandra").
        options(Map( "table" -> "potential_fraud_per_state", "keyspace" -> "retail")).
        mode(SaveMode.Overwrite).
        save()

## RDD based on the store table

In [ ]:
   val stores = sc.cassandraTable("retail","stores").select("store_id","address",
      "address_2","address_3","city","state","zip","size_in_sf"
    ).as(Store)


In [ ]:
stores.take(2)

## Another RDD based on receipts.  
### No case class as it's only two columns

In [ ]:
val receipts = sc.cassandraTable("retail","receipts_by_store_date")

In [ ]:
import scala.math.BigDecimal.RoundingMode

val total_receipts_by_store = receipts.map(r => (r.getInt("store_id"), r.getDecimal("receipt_total").setScale(2,RoundingMode.HALF_EVEN) )  ).reduceByKey(_+_)   // Add up by store

In [ ]:
total_receipts_by_store take 10 foreach println